# 03. YOLOv8n-pose モデル学習

## 概要
1. YOLOv8n-poseの事前学習済みモデルをベースにファインチューニング
2. 学習済みモデルをONNX形式にエクスポート
3. モデルアーティファクトをS3に保存

## 1. セットアップ

In [ ]:
# 依存パッケージのインストール
!pip install ultralytics onnx onnxruntime --quiet

In [ ]:
import boto3
import json
import os
import shutil
from pathlib import Path
from datetime import datetime

import sagemaker
from ultralytics import YOLO

s3 = boto3.client('s3')
sess = sagemaker.Session()
SAGEMAKER_BUCKET = sess.default_bucket()

# パス設定
DATASET_DIR = Path('/tmp/yolo_dataset')
DATASET_YAML = DATASET_DIR / 'dataset.yaml'
OUTPUT_DIR = Path('/tmp/yolo_output')
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

print(f'Dataset: {DATASET_DIR}')
print(f'Output: {OUTPUT_DIR}')
print(f'S3 Bucket: {SAGEMAKER_BUCKET}')

## 2. データセットの確認

02_annotation_to_yolo.ipynbで作成したデータセットを確認する。
存在しない場合はS3からダウンロードする。

In [ ]:
import subprocess

# ローカルにデータセットがない場合はS3からダウンロード
if not DATASET_YAML.exists():
    print('ローカルにデータセットが見つかりません。S3からダウンロードします...')
    S3_DATASET_URI = f's3://{SAGEMAKER_BUCKET}/yolo-pose/dataset'
    cmd = f'aws s3 sync {S3_DATASET_URI} {DATASET_DIR} --quiet'
    result = subprocess.run(cmd, shell=True, capture_output=True, text=True)
    if result.returncode != 0:
        print(f'ダウンロードエラー: {result.stderr}')
    else:
        print('ダウンロード完了')

# 確認
print(f'\ndataset.yaml 存在: {DATASET_YAML.exists()}')
if DATASET_YAML.exists():
    print(DATASET_YAML.read_text())

for split in ['train', 'val', 'test']:
    img_dir = DATASET_DIR / 'images' / split
    if img_dir.exists():
        count = len(list(img_dir.glob('*.*')))
        print(f'{split}: {count} images')

## 3. YOLOv8n-pose モデル学習

### ハイパーパラメータ
| パラメータ | 値 | 説明 |
|-----------|-----|------|
| model | yolov8n-pose.pt | 軽量版poseモデル |
| epochs | 100 | エポック数 |
| imgsz | 640 | 入力画像サイズ |
| batch | 16 | バッチサイズ |
| patience | 20 | Early stopping patience |
| lr0 | 0.01 | 初期学習率 |
| lrf | 0.01 | 最終学習率係数 |

In [ ]:
# ハイパーパラメータ
EPOCHS = 100
IMAGE_SIZE = 640
BATCH_SIZE = 16
PATIENCE = 20
LR0 = 0.01
LRF = 0.01

# モデルロード（事前学習済み）
model = YOLO('yolov8n-pose.pt')

print(f'モデル: yolov8n-pose')
print(f'パラメータ数: {sum(p.numel() for p in model.model.parameters()):,}')

In [ ]:
# 学習実行
results = model.train(
    data=str(DATASET_YAML),
    epochs=EPOCHS,
    imgsz=IMAGE_SIZE,
    batch=BATCH_SIZE,
    patience=PATIENCE,
    lr0=LR0,
    lrf=LRF,
    project=str(OUTPUT_DIR),
    name='meter_keypoint',
    exist_ok=True,
    # データ拡張
    hsv_h=0.015,
    hsv_s=0.7,
    hsv_v=0.4,
    degrees=15.0,     # 回転（メーターの傾き対応）
    translate=0.1,
    scale=0.3,
    flipud=0.0,       # 上下反転は無効（メーターは向き固定）
    fliplr=0.0,       # 左右反転も無効
    mosaic=0.5,
)

print('\n学習完了')

## 4. 学習結果の確認

In [ ]:
# 学習結果ディレクトリ
train_dir = OUTPUT_DIR / 'meter_keypoint'

# best.pt の存在確認
best_pt = train_dir / 'weights' / 'best.pt'
last_pt = train_dir / 'weights' / 'last.pt'

print(f'best.pt 存在: {best_pt.exists()}')
if best_pt.exists():
    print(f'best.pt サイズ: {best_pt.stat().st_size / 1024 / 1024:.1f} MB')

print(f'last.pt 存在: {last_pt.exists()}')

# results.csv の確認
results_csv = train_dir / 'results.csv'
if results_csv.exists():
    import csv
    with open(results_csv) as f:
        reader = csv.DictReader(f)
        rows = list(reader)
        if rows:
            last_row = rows[-1]
            print(f'\n最終エポック結果:')
            for key, val in last_row.items():
                key = key.strip()
                if key:
                    print(f'  {key}: {val}')

In [ ]:
# 学習曲線の画像を表示
from IPython.display import Image as IPImage, display

results_img = train_dir / 'results.png'
if results_img.exists():
    display(IPImage(filename=str(results_img)))
else:
    print('results.png が見つかりません')

## 5. テストデータでの評価

In [ ]:
# best.pt をロードして評価
best_model = YOLO(str(best_pt))

# testデータで評価
test_results = best_model.val(
    data=str(DATASET_YAML),
    split='test',
    imgsz=IMAGE_SIZE,
    batch=BATCH_SIZE
)

print(f'\nテスト結果:')
print(f'  mAP50: {test_results.box.map50:.4f}')
print(f'  mAP50-95: {test_results.box.map:.4f}')
if hasattr(test_results, 'pose'):
    print(f'  Pose mAP50: {test_results.pose.map50:.4f}')
    print(f'  Pose mAP50-95: {test_results.pose.map:.4f}')

## 6. ONNX形式にエクスポート

サーバーレス推論エンドポイントではONNXランタイムを使用するため、best.ptをONNXに変換する。

In [ ]:
# ONNX エクスポート
onnx_path = best_model.export(
    format='onnx',
    imgsz=IMAGE_SIZE,
    simplify=True,
    opset=17
)

print(f'\nONNXモデル: {onnx_path}')
print(f'サイズ: {Path(onnx_path).stat().st_size / 1024 / 1024:.1f} MB')

In [ ]:
# ONNX モデルの動作確認
import onnxruntime as ort

ort_session = ort.InferenceSession(str(onnx_path))

# 入出力の確認
print('入力:')
for inp in ort_session.get_inputs():
    print(f'  {inp.name}: {inp.shape} ({inp.type})')

print('\n出力:')
for out in ort_session.get_outputs():
    print(f'  {out.name}: {out.shape} ({out.type})')

## 7. モデルアーティファクトをS3に保存

SageMakerサーバーレスエンドポイント用に `model.tar.gz` を作成する。

In [ ]:
import tarfile

# model.tar.gz の構造:
# model.tar.gz/
# ├── best.onnx         # ONNXモデル
# └── model_config.json  # モデル設定（キーポイント定義等）

model_config = {
    'model_type': 'yolov8n-pose',
    'input_size': IMAGE_SIZE,
    'num_keypoints': 4,
    'keypoint_labels': ['needle_tip', 'needle_center', 'scale_min', 'scale_max'],
    'class_names': ['meter'],
    'created_at': datetime.now().isoformat(),
    'training_epochs': EPOCHS,
    'training_batch_size': BATCH_SIZE
}

# model_config.json を保存
config_path = OUTPUT_DIR / 'model_config.json'
config_path.write_text(json.dumps(model_config, indent=2))

# tar.gz 作成
tarball_path = OUTPUT_DIR / 'model.tar.gz'
with tarfile.open(str(tarball_path), 'w:gz') as tar:
    tar.add(str(onnx_path), arcname='best.onnx')
    tar.add(str(config_path), arcname='model_config.json')

print(f'model.tar.gz: {tarball_path}')
print(f'サイズ: {tarball_path.stat().st_size / 1024 / 1024:.1f} MB')

In [ ]:
# S3にアップロード
timestamp = datetime.now().strftime('%Y%m%d-%H%M%S')
model_s3_key = f'yolo-pose/models/{timestamp}/model.tar.gz'

s3.upload_file(
    str(tarball_path),
    SAGEMAKER_BUCKET,
    model_s3_key
)

MODEL_S3_URI = f's3://{SAGEMAKER_BUCKET}/{model_s3_key}'
print(f'モデルアップロード完了: {MODEL_S3_URI}')

In [ ]:
# 学習結果（ログ、画像）もS3に保存
results_s3_prefix = f'yolo-pose/training-results/{timestamp}'
cmd = f'aws s3 sync {train_dir} s3://{SAGEMAKER_BUCKET}/{results_s3_prefix} --quiet'
result = subprocess.run(cmd, shell=True, capture_output=True, text=True)

if result.returncode == 0:
    print(f'学習結果アップロード完了: s3://{SAGEMAKER_BUCKET}/{results_s3_prefix}')
else:
    print(f'エラー: {result.stderr}')

## 8. 学習結果サマリー

In [ ]:
print('=' * 60)
print('学習結果サマリー')
print('=' * 60)
print(f'モデル: yolov8n-pose (fine-tuned)')
print(f'エポック: {EPOCHS}')
print(f'画像サイズ: {IMAGE_SIZE}x{IMAGE_SIZE}')
print(f'ONNX モデルサイズ: {Path(onnx_path).stat().st_size / 1024 / 1024:.1f} MB')
print(f'model.tar.gz サイズ: {tarball_path.stat().st_size / 1024 / 1024:.1f} MB')
print(f'\nS3 モデルパス: {MODEL_S3_URI}')
print(f'S3 学習結果: s3://{SAGEMAKER_BUCKET}/{results_s3_prefix}')
print('=' * 60)
print('\n次のステップ: 04_deploy_serverless.ipynb でサーバーレスエンドポイントをデプロイ')

## 次のステップ

**04_deploy_serverless.ipynb** で以下を実行:
- カスタム推論コード（inference.py）の作成
- SageMakerモデルの登録
- サーバーレス推論エンドポイントのデプロイ

### 使用する情報
- モデルS3パス: `MODEL_S3_URI` の値をコピー